# Understanding joblib multiprocess

## Parallel barcktesting

In [11]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ubuntu/varios/skforecast'

In [12]:
import numpy as np
import pandas as pd
import time

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg
import skforecast
%load_ext pyinstrument 
print(skforecast.__version__)

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument
0.9.0


In [13]:
forecaster = ForecasterAutoreg(regressor=HistGradientBoostingRegressor(random_state=666), lags=50)
forecaster = ForecasterAutoreg(regressor=LGBMRegressor(random_state=666), lags=50)
#forecaster = ForecasterAutoreg(regressor=LinearRegression(), lags=50)
n=10_000
rgn = np.random.default_rng(seed=123)
y = pd.Series(rgn.random(size=(n)), name="y")
exog = pd.DataFrame(rgn.random(size=(n, 10)))
exog.columns = [f"col_{i}" for i in range(exog.shape[1])]
y_train = y[:-int(n/2)]
y

0       0.682352
1       0.053821
2       0.220360
3       0.184372
4       0.175906
          ...   
9995    0.344087
9996    0.517366
9997    0.502259
9998    0.370318
9999    0.408432
Name: y, Length: 10000, dtype: float64

In [14]:
# Backtesting with refit
# ==============================================================================
metric, backtest_predictions = backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = exog,
                                    initial_train_size  = len(y_train),
                                    fixed_train_size    = False,
                                    steps               = 24,
                                    metric              = 'mean_squared_error',
                                    refit               = True,
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    verbose             = False
                            )
print(metric)
backtest_predictions

  0%|          | 0/209 [00:00<?, ?it/s]

/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:427: LongTrainingWarning: The forecaster will be fit 209 times. This can take substantial amounts of time. If not feasible, try with `refit = False`.
 
 You can suppress this warning using: warnings.simplefilter('ignore', category=LongTrainingWarning)
  warnings.warn(


0.08526759184414298


,pred
5000,0.452285
5001,0.505726
5002,0.478823
5003,0.489827
5004,0.465579
...,...
9995,0.424152
9996,0.557427
9997,0.477510
9998,0.477689


In [6]:
# Backtesting without refit
# ==============================================================================
metric, backtest_predictions = backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = exog,
                                    initial_train_size  = len(y_train),
                                    fixed_train_size    = False,
                                    steps               = 24,
                                    metric              = 'mean_squared_error',
                                    refit               = False,
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    verbose             = False
                               )
print(metric)
backtest_predictions

  0%|          | 0/209 [00:00<?, ?it/s]

0.08613453504170875


,pred
5000,0.547143
5001,0.492890
5002,0.439208
5003,0.514884
5004,0.514389
...,...
9995,0.467167
9996,0.546763
9997,0.467555
9998,0.489330


In [15]:
# GridSearch without refit
# ==============================================================================

# Lags used as predictors
lags_grid = [50, 50, 50]

# Regressor hyperparameters
param_grid = {
    'max_iter': [50],
    'max_depth': [5, 10, 15],
#     'min_samples_leaf': [5, 10, 20],
#     'l2_regularization': [0.0, 0.1, 0.5]
}

results_grid = grid_search_forecaster(
                    forecaster          = forecaster,
                    y                   = y,
                    exog                = exog,
                    initial_train_size  = len(y_train),
                    steps               = 24,
                    param_grid         = param_grid,
                    lags_grid          = lags_grid,
                    refit              = False,
                    metric             = 'mean_squared_error',
                    fixed_train_size   = False,
                    return_best        = False,
                    verbose            = False,
                    show_progress      = True
               )

Number of models compared: 9.


lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/3 [00:00<?, ?it/s]

[LightGBM] [Warning] num_iterations is set=50, max_iter=50 will be ignored. Current value: num_iterations=50


/home/ubuntu/anaconda3/envs/skforecast_py10/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `max_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/ubuntu/anaconda3/envs/skforecast_py10/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `max_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/ubuntu/anaconda3/envs/skforecast_py10/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `max_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/ubuntu/anaconda3/envs/skforecast_py10/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `max_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/ub

In [9]:
# Backtesting with refit
# ==============================================================================

# Lags used as predictors
lags_grid = [50, 50, 50]

# Regressor hyperparameters
param_grid = {
    'max_iter': [100, 200],
    'max_depth': [5, 10],
    'min_samples_leaf': [5, 10, 20]
}

start_time = time.time()
results_grid = grid_search_forecaster(
                    forecaster          = forecaster,
                    y                   = y,
                    exog                = exog,
                    initial_train_size  = len(y_train),
                    steps               = 24,
                    param_grid         = param_grid,
                    lags_grid          = lags_grid,
                    refit              = True,
                    metric             = 'mean_squared_error',
                    fixed_train_size   = False,
                    return_best        = False,
                    verbose            = False,
                    show_progress      = True
               )
print(time.time() - start_time)
results_grid

Number of models compared: 3.


lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/1 [00:00<?, ?it/s]

/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:427: LongTrainingWarning: The forecaster will be fit 209 times. This can take substantial amounts of time. If not feasible, try with `refit = False`.
 
 You can suppress this warning using: warnings.simplefilter('ignore', category=LongTrainingWarning)
  warnings.warn(
/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:427: LongTrainingWarning: The forecaster will be fit 209 times. This can take substantial amounts of time. If not feasible, try with `refit = False`.
 
 You can suppress this warning using: warnings.simplefilter('ignore', category=LongTrainingWarning)
  warnings.warn(
/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:427: LongTrainingWarning: The forecaster will be fit 209 times. This can take substantial amounts of time. If not feasible, try with `refit = False`.
 
 You can suppress this warning using: warnings.simplefilter('ignore', category=

99.30256509780884


,lags,params,mean_squared_error
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{},0.083649
1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{},0.083649
2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{},0.083649
